In [1]:
%load_ext autoreload
%autoreload 2

import torch as t
import numpy as np

from memory_module import TGNPLMemory
from msg_func import TGNPLMessage
from msg_agg import *

from neighbor_loader import LastNeighborLoader, LastNeighborLoaderTGNPL

# Test TGNPLMemory

In [2]:
num_nodes = 10
num_prods = 2
raw_msg_dim = 1
state_dim = 10
time_dim = 1
message_module = TGNPLMessage(raw_msg_dim, state_dim+num_prods, time_dim)
aggregator_module = MeanAggregator()

In [3]:
# test initialization
mem = TGNPLMemory(num_nodes,
        num_prods,
        raw_msg_dim,
        state_dim,
        time_dim,
        message_module,
        aggregator_module,
        state_updater_cell="gru",
        use_inventory=True,
        debt_penalty=10,
        consumption_reward=5,
        debug=True)

38 10


In [4]:
# before any interactions have been added
np.random.seed(0)
n_id = np.random.choice(num_nodes, size=5, replace=False)
n_id = t.from_numpy(n_id)
print(n_id)
memory, last_update, loss = mem(n_id)

tensor([2, 8, 4, 9, 1])
msg_s tensor([], size=(0, 38), grad_fn=<ReluBackward0>)
msg_d tensor([], size=(0, 38), grad_fn=<ReluBackward0>)
msg_p tensor([], size=(0, 38), grad_fn=<ReluBackward0>)
aggr tensor([[0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0

In [5]:
last_update

tensor([0, 0, 0, 0, 0])

In [6]:
# try adding interactions
# expectation: 
# 1) get_updated_memory will print 6 nodes, but memories shouldn't be updated yet
# 2) _update_msg_store should update all three msg stores
src = t.Tensor([0, 0, 1, 2]).long()
dst = t.Tensor([3, 3, 3, 0]).long()
prod = t.Tensor([8, 8, 8, 9]).long()
time = t.Tensor(np.ones(4)).long()
raw_msg = t.Tensor([10, 20, 5, 13]).reshape(-1, 1)
mem.update_state(src, dst, prod, time, raw_msg)

msg_s tensor([], size=(0, 38), grad_fn=<ReluBackward0>)
msg_d tensor([], size=(0, 38), grad_fn=<ReluBackward0>)
msg_p tensor([], size=(0, 38), grad_fn=<ReluBackward0>)
aggr tensor([[0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0

In [7]:
mem.msg_s_store

{0: (tensor([0, 0]),
  tensor([3, 3]),
  tensor([8, 8]),
  tensor([1, 1]),
  tensor([[10.],
          [20.]])),
 1: (tensor([1]), tensor([3]), tensor([8]), tensor([1]), tensor([[5.]])),
 2: (tensor([2]), tensor([0]), tensor([9]), tensor([1]), tensor([[13.]])),
 3: (tensor([], dtype=torch.int64),
  tensor([], dtype=torch.int64),
  tensor([], dtype=torch.int64),
  tensor([], dtype=torch.int64),
  tensor([], size=(0, 1))),
 4: (tensor([], dtype=torch.int64),
  tensor([], dtype=torch.int64),
  tensor([], dtype=torch.int64),
  tensor([], dtype=torch.int64),
  tensor([], size=(0, 1))),
 5: (tensor([], dtype=torch.int64),
  tensor([], dtype=torch.int64),
  tensor([], dtype=torch.int64),
  tensor([], dtype=torch.int64),
  tensor([], size=(0, 1))),
 6: (tensor([], dtype=torch.int64),
  tensor([], dtype=torch.int64),
  tensor([], dtype=torch.int64),
  tensor([], dtype=torch.int64),
  tensor([], size=(0, 1))),
 7: (tensor([], dtype=torch.int64),
  tensor([], dtype=torch.int64),
  tensor([], dtype

In [8]:
mem.msg_d_store

{0: (tensor([2]), tensor([0]), tensor([9]), tensor([1]), tensor([[13.]])),
 1: (tensor([], dtype=torch.int64),
  tensor([], dtype=torch.int64),
  tensor([], dtype=torch.int64),
  tensor([], dtype=torch.int64),
  tensor([], size=(0, 1))),
 2: (tensor([], dtype=torch.int64),
  tensor([], dtype=torch.int64),
  tensor([], dtype=torch.int64),
  tensor([], dtype=torch.int64),
  tensor([], size=(0, 1))),
 3: (tensor([0, 0, 1]),
  tensor([3, 3, 3]),
  tensor([8, 8, 8]),
  tensor([1, 1, 1]),
  tensor([[10.],
          [20.],
          [ 5.]])),
 4: (tensor([], dtype=torch.int64),
  tensor([], dtype=torch.int64),
  tensor([], dtype=torch.int64),
  tensor([], dtype=torch.int64),
  tensor([], size=(0, 1))),
 5: (tensor([], dtype=torch.int64),
  tensor([], dtype=torch.int64),
  tensor([], dtype=torch.int64),
  tensor([], dtype=torch.int64),
  tensor([], size=(0, 1))),
 6: (tensor([], dtype=torch.int64),
  tensor([], dtype=torch.int64),
  tensor([], dtype=torch.int64),
  tensor([], dtype=torch.int64

In [9]:
mem.msg_p_store

{0: (tensor([], dtype=torch.int64),
  tensor([], dtype=torch.int64),
  tensor([], dtype=torch.int64),
  tensor([], dtype=torch.int64),
  tensor([], size=(0, 1))),
 1: (tensor([], dtype=torch.int64),
  tensor([], dtype=torch.int64),
  tensor([], dtype=torch.int64),
  tensor([], dtype=torch.int64),
  tensor([], size=(0, 1))),
 2: (tensor([], dtype=torch.int64),
  tensor([], dtype=torch.int64),
  tensor([], dtype=torch.int64),
  tensor([], dtype=torch.int64),
  tensor([], size=(0, 1))),
 3: (tensor([], dtype=torch.int64),
  tensor([], dtype=torch.int64),
  tensor([], dtype=torch.int64),
  tensor([], dtype=torch.int64),
  tensor([], size=(0, 1))),
 4: (tensor([], dtype=torch.int64),
  tensor([], dtype=torch.int64),
  tensor([], dtype=torch.int64),
  tensor([], dtype=torch.int64),
  tensor([], size=(0, 1))),
 5: (tensor([], dtype=torch.int64),
  tensor([], dtype=torch.int64),
  tensor([], dtype=torch.int64),
  tensor([], dtype=torch.int64),
  tensor([], size=(0, 1))),
 6: (tensor([], dtype=

In [10]:
# now get memory again - only nodes with interactions should've changed
n_id = t.from_numpy(np.arange(num_nodes))
memory, last_update, loss = mem(n_id)  # test .forward()

msg_s tensor([[0.0633, 0.0000, 0.9071, 0.6209, 1.2335, 0.6883, 0.1432, 0.6888, 0.5723,
         0.9815, 0.0000, 1.1155, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.3108,
         1.1213, 0.0000, 1.1136, 1.1041, 0.3739, 0.6850, 0.0000, 0.5282, 0.0000,
         0.0000, 1.1387, 0.0000, 0.1244, 0.0000, 0.8950, 0.0000, 0.0000, 1.0217,
         1.3443, 0.0000],
        [0.0615, 0.0000, 0.8504, 0.6579, 1.2221, 0.7446, 0.0964, 0.7914, 0.5253,
         0.9308, 0.0000, 1.0441, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.3535,
         1.1436, 0.0000, 1.1878, 1.1299, 0.4242, 0.6711, 0.0000, 0.4883, 0.0000,
         0.0000, 1.1388, 0.0000, 0.0706, 0.0000, 0.8672, 0.0000, 0.0000, 1.0399,
         1.3335, 0.0000],
        [0.0665, 0.0000, 1.0163, 0.5414, 1.2482, 0.5690, 0.2373, 0.4754, 0.6640,
         1.0779, 0.0000, 1.2530, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.2219,
         1.0681, 0.0000, 0.9551, 1.0439, 0.2691, 0.7086, 0.0000, 0.6058, 0.0000,
         0.0000, 1.1308, 0.0000, 0.2331, 0.0000, 0.

In [11]:
# 2 and 9, same state, different inventory
# 2 supplied product 9
# product 9 has no inventory
memory[[2,9]]

tensor([[-0.4671,  0.2658, -0.2437,  0.6124,  0.2226,  0.6505, -0.2939, -0.6747,
         -0.3364, -0.3542, -1.3207, -1.3207],
        [-0.4671,  0.2658, -0.2437,  0.6124,  0.2226,  0.6505, -0.2939, -0.6747,
         -0.3364, -0.3542,  0.0000,  0.0000]], grad_fn=<IndexBackward0>)

In [12]:
# 3 and 8, same state, different inventory
# 3 received exactly 35 of product 8
# product 8 has no inventory
memory[[3,8]]

tensor([[-0.4591,  0.2596, -0.2281,  0.6266,  0.2568,  0.6486, -0.3022, -0.6668,
         -0.3243, -0.3549, 35.0000,  0.0000],
        [-0.4591,  0.2596, -0.2281,  0.6266,  0.2568,  0.6486, -0.3022, -0.6668,
         -0.3243, -0.3549,  0.0000,  0.0000]], grad_fn=<IndexBackward0>)

In [13]:
# should be unaffected
memory[[4,5,6,7]]

tensor([[-0.1104, -0.1008, -0.0982,  0.0498,  0.1770,  0.0185,  0.0219, -0.1343,
         -0.0609, -0.0813,  0.0000,  0.0000],
        [-0.1104, -0.1008, -0.0982,  0.0498,  0.1770,  0.0185,  0.0219, -0.1343,
         -0.0609, -0.0813,  0.0000,  0.0000],
        [-0.1104, -0.1008, -0.0982,  0.0498,  0.1770,  0.0185,  0.0219, -0.1343,
         -0.0609, -0.0813,  0.0000,  0.0000],
        [-0.1104, -0.1008, -0.0982,  0.0498,  0.1770,  0.0185,  0.0219, -0.1343,
         -0.0609, -0.0813,  0.0000,  0.0000]], grad_fn=<IndexBackward0>)

In [14]:
# should only be updated for nodes in transactions
last_update

tensor([1, 1, 1, 1, 0, 0, 0, 0, 1, 1])

## Test attention weight learning

In [15]:
num_nodes = 6
num_prods = 3
raw_msg_dim = 1
state_dim = 2
time_dim = 1
message_module = TGNPLMessage(raw_msg_dim, state_dim+num_prods, time_dim)
aggregator_module = MeanAggregator()

In [16]:
# raw_msg_dim + (3 * memory_dim) + time_dim
message_module.out_channels

17

In [17]:
n_id = t.arange(0, num_nodes).long()
print(n_id)

tensor([0, 1, 2, 3, 4, 5])


In [23]:
# test directly learning attention weights
mem = TGNPLMemory(num_nodes,
        num_prods,
        raw_msg_dim,
        state_dim,
        time_dim,
        message_module,
        aggregator_module,
        state_updater_cell="gru",
        use_inventory=True,
        learn_att_direct=True,
        debt_penalty=10,
        consumption_reward=1,
        debug=False)
opt = t.optim.Adam(mem.parameters(), lr=1e-2)
for name, param in mem.named_parameters():
    if param.requires_grad:
        print(name, param.data.shape)

17 2
att_weights torch.Size([3, 3])
msg_s_module.lin.weight torch.Size([17, 17])
msg_s_module.lin.bias torch.Size([17])
msg_s_module.layer_norm.weight torch.Size([17])
msg_s_module.layer_norm.bias torch.Size([17])
msg_d_module.lin.weight torch.Size([17, 17])
msg_d_module.lin.bias torch.Size([17])
msg_d_module.layer_norm.weight torch.Size([17])
msg_d_module.layer_norm.bias torch.Size([17])
msg_p_module.lin.weight torch.Size([17, 17])
msg_p_module.lin.bias torch.Size([17])
msg_p_module.layer_norm.weight torch.Size([17])
msg_p_module.layer_norm.bias torch.Size([17])
time_enc.lin.weight torch.Size([1, 1])
time_enc.lin.bias torch.Size([1])
state_updater.weight_ih torch.Size([6, 17])
state_updater.weight_hh torch.Size([6, 2])
state_updater.bias_ih torch.Size([6])
state_updater.bias_hh torch.Size([6])


In [24]:
# need products 3 and 4 to make 5
# att weights should have nonzero in 3,1 and/or 3,2 - this works
t.autograd.set_detect_anomaly(True)
for i in range(1, 31):
    opt.zero_grad()
    if (i % 2) == 0:
        # 1 sells 5 to 2
        src = t.Tensor([1]).long()
        dst = t.Tensor([2]).long()
        prod = t.Tensor([5]).long()
        time = t.Tensor([i]).long()
        raw_msg = t.Tensor([1]).reshape(-1, 1)
    else:
        # 1 buys 3 and 4 from 0
        src = t.Tensor([0, 0]).long()
        dst = t.Tensor([1, 1]).long()
        prod = t.Tensor([3, 4]).long()
        time = t.Tensor([i, i]).long()
        raw_msg = t.Tensor([2, 4]).reshape(-1, 1)

    print('iter', i)
    mem.update_state(src, dst, prod, time, raw_msg)
    memory, last_update, loss = mem(n_id)
    print('loss', loss)
    att_weights = mem.get_prod_attention()
    print('att weights', att_weights)
    loss.backward()
    opt.step()
    mem.detach()

iter 1
loss tensor(162., grad_fn=<SumBackward0>)
att weights tensor([[1., 1., 1.],
        [1., 1., 1.],
        [1., 1., 1.]], grad_fn=<ReluBackward0>)
iter 2
loss tensor(167.3800, grad_fn=<SumBackward0>)
att weights tensor([[0.9900, 0.9900, 0.9900],
        [0.9900, 0.9900, 0.9900],
        [1.0000, 1.0000, 1.0000]], grad_fn=<ReluBackward0>)
iter 3
loss tensor(352.0037, grad_fn=<SumBackward0>)
att weights tensor([[0.9800, 0.9800, 0.9800],
        [0.9800, 0.9800, 0.9800],
        [1.0074, 1.0074, 0.9926]], grad_fn=<ReluBackward0>)
iter 4
loss tensor(360.1991, grad_fn=<SumBackward0>)
att weights tensor([[0.9704, 0.9704, 0.9704],
        [0.9704, 0.9704, 0.9704],
        [1.0160, 1.0160, 0.9843]], grad_fn=<ReluBackward0>)
iter 5
loss tensor(541.2927, grad_fn=<SumBackward0>)
att weights tensor([[0.9609, 0.9609, 0.9609],
        [0.9609, 0.9609, 0.9609],
        [1.0251, 1.0251, 0.9754]], grad_fn=<ReluBackward0>)
iter 6
loss tensor(549.2872, grad_fn=<SumBackward0>)
att weights tensor([[0

In [25]:
# test learning attention weights with product memories
mem = TGNPLMemory(num_nodes,
        num_prods,
        raw_msg_dim,
        state_dim,
        time_dim,
        message_module,
        aggregator_module,
        state_updater_cell="gru",
        use_inventory=True,
        learn_att_direct=False,
        debt_penalty=10,
        consumption_reward=1,
        debug=False)
opt = t.optim.Adam(mem.parameters(), lr=1e-2)
for name, param in mem.named_parameters():
    if param.requires_grad:
        print(name, param.data.shape)

17 2
prod_bilinear torch.Size([2, 2])
msg_s_module.lin.weight torch.Size([17, 17])
msg_s_module.lin.bias torch.Size([17])
msg_s_module.layer_norm.weight torch.Size([17])
msg_s_module.layer_norm.bias torch.Size([17])
msg_d_module.lin.weight torch.Size([17, 17])
msg_d_module.lin.bias torch.Size([17])
msg_d_module.layer_norm.weight torch.Size([17])
msg_d_module.layer_norm.bias torch.Size([17])
msg_p_module.lin.weight torch.Size([17, 17])
msg_p_module.lin.bias torch.Size([17])
msg_p_module.layer_norm.weight torch.Size([17])
msg_p_module.layer_norm.bias torch.Size([17])
time_enc.lin.weight torch.Size([1, 1])
time_enc.lin.bias torch.Size([1])
state_updater.weight_ih torch.Size([6, 17])
state_updater.weight_hh torch.Size([6, 2])
state_updater.bias_ih torch.Size([6])
state_updater.bias_hh torch.Size([6])


In [26]:
# need products 3 and 4 to make 5
# att weights should have nonzero in 3,1 and/or 3,2 - this doesn't work
t.autograd.set_detect_anomaly(True)
for i in range(1, 31):
    opt.zero_grad()
    if (i % 2) == 0:
        # 1 sells 5 to 2
        src = t.Tensor([1]).long()
        dst = t.Tensor([2]).long()
        prod = t.Tensor([5]).long()
        time = t.Tensor([i]).long()
        raw_msg = t.Tensor([1]).reshape(-1, 1)
    else:
        # 1 buys 3 and 4 from 0
        src = t.Tensor([0, 0]).long()
        dst = t.Tensor([1, 1]).long()
        prod = t.Tensor([3, 4]).long()
        time = t.Tensor([i, i]).long()
        raw_msg = t.Tensor([2, 4]).reshape(-1, 1)

    print('iter', i)
    mem.update_state(src, dst, prod, time, raw_msg)
    memory, last_update, loss = mem(n_id)
    print('loss', loss)
    att_weights = mem.get_prod_attention()
    print('att weights', att_weights)
    loss.backward()
    opt.step()
    mem.detach()

iter 1
loss tensor(0.4768, grad_fn=<SumBackward0>)
att weights tensor([[0.0044, 0.0044, 0.0000],
        [0.0044, 0.0044, 0.0000],
        [0.0000, 0.0000, 0.0000]], grad_fn=<ReluBackward0>)
iter 2
loss tensor(0.4148, grad_fn=<SumBackward0>)
att weights tensor([[0.0031, 0.0031, 0.0015],
        [0.0031, 0.0031, 0.0015],
        [0.0015, 0.0015, 0.0005]], grad_fn=<ReluBackward0>)
iter 3
loss tensor(3.2606, grad_fn=<SumBackward0>)
att weights tensor([[0.1207, 0.0161, 0.0000],
        [0.0161, 0.0000, 0.0000],
        [0.0000, 0.0000, 0.0000]], grad_fn=<ReluBackward0>)
iter 4
loss tensor(3.2100, grad_fn=<SumBackward0>)
att weights tensor([[0.1169, 0.0131, 0.0000],
        [0.0131, 0.0000, 0.0000],
        [0.0000, 0.0000, 0.0202]], grad_fn=<ReluBackward0>)
iter 5
loss tensor(15.2349, grad_fn=<SumBackward0>)
att weights tensor([[0.1803, 0.1137, 0.0000],
        [0.1137, 0.0697, 0.0000],
        [0.0000, 0.0000, 0.0142]], grad_fn=<ReluBackward0>)
iter 6
loss tensor(14.6644, grad_fn=<SumBack

# Test Neighbor Loader

In [2]:
neighbor_loader = LastNeighborLoaderTGNPL(9, size=2)

In [3]:
# Test init
print(neighbor_loader.neighbors.shape)
print(neighbor_loader.e_id.shape)
print(neighbor_loader._assoc.shape)
self = neighbor_loader

torch.Size([9, 2])
torch.Size([9, 2])
torch.Size([9])


In [7]:
# Test insert
src = torch.Tensor([0, 1, 2]).to(torch.long)
dst = torch.Tensor([3, 4, 5]).to(torch.long)
prod = torch.Tensor([6, 7, 8]).to(torch.long)

nodes = torch.cat([prod, src, prod, dst], dim=0)
n_id = nodes.unique()
neighbor_loader.insert(src, dst, prod)

print(n_id, self.neighbors[n_id]) # This is correct

tensor([0, 1, 2, 3, 4, 5, 6, 7, 8]) tensor([[              6, 159321811882214],
        [              7, 158772056065462],
        [              8,  88214333292640],
        [              6,       110377216],
        [              7,               0],
        [              8,       112948880],
        [              3,               0],
        [              4,               1],
        [              5,               2]])


In [8]:
# Test _call_
f_id = torch.cat([src, dst]).unique()
p_id = torch.cat([prod]).unique()

n_id, edge_index, e_id = neighbor_loader(f_id, p_id)

# Ground truth: 6 edges 0-6, 3-6, 1-7, 4-7, 2-8, 5-8
print(n_id, n_id.shape)
print(edge_index, edge_index.shape)
print(e_id, e_id.shape)

tensor([0, 1, 2, 3, 4, 5, 6, 7, 8]) torch.Size([9])
tensor([[6, 7, 8, 6, 7, 8, 3, 0, 4, 1, 5, 2],
        [0, 1, 2, 3, 4, 5, 6, 6, 7, 7, 8, 8]]) torch.Size([2, 12])
tensor([0, 1, 2, 3, 4, 5, 3, 0, 4, 1, 5, 2]) torch.Size([12])


In [9]:
# Test insert
src = torch.Tensor([0]).to(torch.long)
dst = torch.Tensor([1]).to(torch.long)
prod = torch.Tensor([6]).to(torch.long)

nodes = torch.cat([prod, src, prod, dst], dim=0)
n_id = nodes.unique()
neighbor_loader.insert(src, dst, prod)

print(n_id, self.neighbors[n_id]) # This is correct

tensor([0, 1, 6]) tensor([[6, 6],
        [6, 7],
        [1, 0]])


In [10]:
# Test _call_
f_id = torch.cat([src, dst]).unique()
p_id = torch.cat([prod]).unique()

n_id, edge_index, e_id = neighbor_loader(f_id, p_id)

# Ground truth: 6 edges 0-6, 1-6, 1-7
print(n_id, n_id.shape)
print(edge_index, edge_index.shape)
print(e_id, e_id.shape)

tensor([0, 1, 6, 7]) torch.Size([4])
tensor([[2, 2, 2, 3, 1, 0],
        [0, 0, 1, 1, 2, 2]]) torch.Size([2, 6])
tensor([6, 0, 7, 1, 7, 6]) torch.Size([6])
